# Analysis of decay energies for $^{137}Cs/^{137m}Ba$ and $^{90}Sr/^{90}Y$ decay chains

In [1]:
import os
import urllib
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile
from tempfile import TemporaryDirectory
import shutil
import yaml

import numpy as np
import pandas as pd

In [2]:
import sandy

The values recommended by DDEP are available for comparison at:

|Nuclide  |url                                                          |
|:--------|:------------------------------------------------------------|
| Cs-137  | http://www.nucleide.org/DDEP_WG/Nuclides/Cs-137_tables.pdf  |
| Ba-137m | http://www.nucleide.org/DDEP_WG/Nuclides/Ba-137m_tables.pdf |
| Sr-90   | http://www.nucleide.org/DDEP_WG/Nuclides/Sr-90_tables.pdf   |
| Y-90    | http://www.nucleide.org/DDEP_WG/Nuclides/Y-90_tables.pdf    |

## Funcions to read decay data using sandy

In [3]:
def read_endfb80_decay_data():
    url = 'https://www.nndc.bnl.gov/endf/b8.0/zips/ENDF-B-VIII.0_decay.zip'
    with TemporaryDirectory(dir=os.getcwd()) as tmpdirname:
        with urlopen(url) as zipresp:
            with ZipFile(BytesIO(zipresp.read())) as zfile:
                zfile.extractall(tmpdirname)

        text = ""
        files = ("dec-038_Sr_090.endf", "dec-039_Y_090.endf", "dec-055_Cs_137.endf", "dec-056_Ba_137m1.endf")
        for filename in files:
            file = os.path.join(tmpdirname, "ENDF-B-VIII.0_decay", filename)
            with open(file) as f:
                text += f.read()

    tape = sandy.Endf6.from_text(text)
    rdd = sandy.DecayData.from_endf6(tape, verbose=True)
    return rdd

In [4]:
def read_endfb71_decay_data():
    url = 'https://www.nndc.bnl.gov/endf/b7.1/zips/ENDF-B-VII.1-decay.zip'
    with TemporaryDirectory(dir=os.getcwd()) as tmpdirname:
        with urlopen(url) as zipresp:
            with ZipFile(BytesIO(zipresp.read())) as zfile:
                zfile.extractall(tmpdirname)

        text = ""
        files = ("dec-038_Sr_090.endf", "dec-039_Y_090.endf", "dec-055_Cs_137.endf", "dec-056_Ba_137m1.endf")
        for filename in files:
            file = os.path.join(tmpdirname, "decay", filename)
            with open(file) as f:
                text += f.read()

    tape = sandy.Endf6.from_text(text)
    rdd = sandy.DecayData.from_endf6(tape, verbose=True)
    return rdd

In [5]:
def read_endfb70_decay_data():
    url = 'https://www.nndc.bnl.gov/endf/b7.0/zips/dec-ENDF-VII0.endf.zip'
    with TemporaryDirectory(dir=os.getcwd()) as tmpdirname:
        with urlopen(url) as zipresp:
            with ZipFile(BytesIO(zipresp.read())) as zfile:
                zfile.extractall(tmpdirname)

        file = os.path.join(tmpdirname, "dec-ENDF-VII0.endf")
        with open(file) as f:
            text = f.read()

    tape = sandy.Endf6.from_text(text)
    listmat = [
        932,    # Sr-90
        973,    # Y-90
        1782,   # Cs-137
        1831,   # Ba-137m
    ]

    rdd = sandy.DecayData.from_endf6(tape.filter_by(listmat=listmat), verbose=True)
    return rdd

In [6]:
def read_jeff_decay_data(url):
    with urllib.request.urlopen(url) as f:
        text = f.read().decode('utf-8')
    tape = sandy.Endf6.from_text(text)
    listmat = [
        965,    # Sr-90
        1004,   # Y-90
        1809,   # Cs-137
        1857,   # Ba-137m
    ]
    rdd = sandy.DecayData.from_endf6(tape.filter_by(listmat=listmat), verbose=True)
    return rdd

In [7]:
def read_jeff33_decay_data():
    url = "https://www.oecd-nea.org/dbdata/jeff/jeff33/downloads/JEFF33-rdd_all.asc"
    return read_jeff_decay_data(url)

In [8]:
def read_jeff311_decay_data():
    url = "https://www.oecd-nea.org/dbforms/data/eva/evatapes/jeff_31/JEFF31/JEFF31RDD.ASC"
    return read_jeff_decay_data(url)

In [9]:
def read_jeff_decay_data(url):
    with urllib.request.urlopen(url) as f:
        text = f.read().decode('utf-8')
    tape = sandy.Endf6.from_text(text)
    listmat = [
        965,    # Sr-90
        1004,   # Y-90
        1809,   # Cs-137
        1857,   # Ba-137m
    ]
    rdd = sandy.DecayData.from_endf6(tape.filter_by(listmat=listmat), verbose=True)
    return rdd

In [10]:
def read_jendl_fpd2011_decay_data():
    urls = {
        "Sr90": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-FPD-2011/038_Sr_090.jfpd11.gz",
        "Y90": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-FPD-2011/039_Y_090.jfpd11.gz",
        "Cs137": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-FPD-2011/055_Cs_137.jfpd11.gz",
        "Ba137m": "https://wwwndc.jaea.go.jp/cgi-bin/UNZIP_jendl.cgi?/JENDL/JENDL-FPD-2011/056_Ba_137M.jfpd11.gz",
    }
    text = ""
    for url in urls.values():
        with urllib.request.urlopen(url) as f:
            text += f.read().decode('utf-8')
    tape = sandy.Endf6.from_text(text)
    rdd = sandy.DecayData.from_endf6(tape, verbose=True)
    return rdd

Decay data are stored as `sandy.DecayData` objects into a dictionary instance called `rdds`. 

In [11]:
rdds = {
    "ENDF/B-VII.0": read_endfb70_decay_data(),
    "ENDF/B-VII.1": read_endfb71_decay_data(),
    "ENDF/B-VIII.0": read_endfb80_decay_data(),
    "JEFF-3.1.1": read_jeff311_decay_data(),
    "JEFF-3.3": read_jeff33_decay_data(),
    "JENDL/FPD-2011": read_jendl_fpd2011_decay_data(),
}

INFO:  reading 'ZAM=380900'...
INFO:  reading 'ZAM=390900'...
INFO:  reading 'ZAM=551370'...
INFO:  reading 'ZAM=561371'...
INFO:  reading 'ZAM=380900'...
INFO:  reading 'ZAM=390900'...
INFO:  reading 'ZAM=551370'...
INFO:  reading 'ZAM=561371'...
INFO:  reading 'ZAM=380900'...
INFO:  reading 'ZAM=390900'...
INFO:  reading 'ZAM=551370'...
INFO:  reading 'ZAM=561371'...
INFO:  reading 'ZAM=380900'...
INFO:  reading 'ZAM=390900'...
INFO:  reading 'ZAM=551370'...
INFO:  reading 'ZAM=561371'...
INFO:  reading 'ZAM=380900'...
INFO:  reading 'ZAM=390900'...
INFO:  reading 'ZAM=551370'...
INFO:  reading 'ZAM=561371'...
INFO:  reading 'ZAM=380900'...
INFO:  reading 'ZAM=390900'...
INFO:  reading 'ZAM=551370'...
INFO:  reading 'ZAM=561371'...


## Data analysis

Let's store the requested information into a dataframe for further processing

In [12]:
entries = []
for lib, rdd in rdds.items():
    for nuclide in rdd.data:
        for decay in ("beta", "gamma", "total"):
            energy = rdd.data[nuclide]['decay_energy'][decay]
            uncertainty = rdd.data[nuclide]['decay_energy_uncertainties'][decay]
            entry = {
                "nuclide": sandy.zam2latex(nuclide),
                "decay": decay,
                "energy": energy * 1e-3,            # in keV
                "uncertainty": uncertainty * 1e-3,  # in keV
                "library": lib,
            }
            entries.append(entry)
data = pd.DataFrame(entries)

In [13]:
def get_table(data, values):
    dfs = {}

    decay = "total"
    dfs[decay] = data.query(f"decay == '{decay}'").pivot_table(index="library", columns="nuclide", values=values)
    dfs[decay].columns = pd.MultiIndex.from_product([[decay], dfs[decay].columns], names=["decay", "nuclide"])

    decay = "beta"
    dfs[decay] = data.query(f"decay == '{decay}'").pivot_table(index="library", columns="nuclide", values=values)
    dfs[decay].columns = pd.MultiIndex.from_product([[decay], dfs[decay].columns], names=["decay", "nuclide"])

    decay = "gamma"
    dfs[decay] = data.query(f"decay == '{decay}'").pivot_table(index="library", columns="nuclide", values=values)
    dfs[decay].columns = pd.MultiIndex.from_product([[decay], dfs[decay].columns], names=["decay", "nuclide"])

    return dfs["beta"].merge(dfs["gamma"], left_index=True, right_index=True)\
                      .merge(dfs["total"], left_index=True, right_index=True)

### Mean recoverable energy values (in keV) per library, nuclide and decay mode

In [14]:
energies = get_table(data, "energy").T
energies

library            ENDF/B-VII.0  ENDF/B-VII.1  ENDF/B-VIII.0  JEFF-3.1.1  \
decay nuclide                                                              
beta  $^{137m}$Ba   6.43140e+01   6.41433e+01    6.41433e+01 6.07298e+01   
      $^{137}$Cs    1.79848e+02   1.79446e+02    1.79446e+02 1.87871e+02   
      $^{90}$Sr     1.95800e+02   1.95800e+02    1.95800e+02 1.74000e+02   
      $^{90}$Y      9.33614e+02   9.33015e+02    9.33015e+02 9.33815e+02   
gamma $^{137m}$Ba   5.98649e+02   5.97253e+02    5.97253e+02 5.98393e+02   
      $^{137}$Cs    1.64430e-03   1.64430e-03    1.64430e-03 1.64430e-03   
      $^{90}$Sr     0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00   
      $^{90}$Y      3.06074e-05   8.85767e-04    8.85767e-04 1.23656e-03   
total $^{137m}$Ba   6.62963e+02   6.61397e+02    6.61397e+02 6.59123e+02   
      $^{137}$Cs    1.79849e+02   1.79448e+02    1.79448e+02 1.87873e+02   
      $^{90}$Sr     1.95800e+02   1.95800e+02    1.95800e+02 1.74000e+02   
      $^{90}$Y      9.33614e+02   9.33016e+02    9.33016e+02 9.33816e+02   

library              JEFF-3.3  JENDL/FPD-2011  
decay nuclide                                  
beta  $^{137m}$Ba 6.07298e+01     6.36400e+01  
      $^{137}$Cs  1.79870e+02     1.87100e+02  
      $^{90}$Sr   1.93500e+02     1.95800e+02  
      $^{90}$Y    9.33815e+02     9.33800e+02  
gamma $^{137m}$Ba 5.98393e+02     5.94800e+02  
      $^{137}$Cs  1.65236e-03     0.00000e+00  
      $^{90}$Sr   0.00000e+00     0.00000e+00  
      $^{90}$Y    1.23656e-03     1.25000e-03  
total $^{137m}$Ba 6.59123e+02     6.58440e+02  
      $^{137}$Cs  1.79872e+02     1.87100e+02  
      $^{90}$Sr   1.93500e+02     1.95800e+02  
      $^{90}$Y    9.33816e+02     9.33801e+02

### Uncertainty on the mean recoverable energy values (in keV) per library, nuclide and decay mode

In absolute units (keV).

In [15]:
uncert = get_table(data, "uncertainty").T
uncert

library            ENDF/B-VII.0  ENDF/B-VII.1  ENDF/B-VIII.0  JEFF-3.1.1  \
decay nuclide                                                              
beta  $^{137m}$Ba   7.34841e-01   7.03199e-01    7.03199e-01 1.78504e+00   
      $^{137}$Cs    6.50395e-01   6.46970e-01    6.46970e-01 9.04425e-01   
      $^{90}$Sr     0.00000e+00   8.00000e-01    8.00000e-01 5.00000e-01   
      $^{90}$Y      1.33276e-02   7.30922e-01    7.30922e-01 1.20015e+00   
gamma $^{137m}$Ba   3.98960e-01   9.27141e-01    9.27141e-01 1.32514e+00   
      $^{137}$Cs    2.26801e-04   2.26801e-04    2.26801e-04 2.26801e-04   
      $^{90}$Sr     0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00   
      $^{90}$Y      6.55873e-06   6.72030e-05    6.72030e-05 1.02541e-04   
total $^{137m}$Ba   8.36158e-01   1.16365e+00    1.16365e+00 2.22314e+00   
      $^{137}$Cs    6.50395e-01   6.46970e-01    6.46970e-01 9.04425e-01   
      $^{90}$Sr     0.00000e+00   8.00000e-01    8.00000e-01 5.00000e-01   
      $^{90}$Y      1.33276e-02   7.30922e-01    7.30922e-01 1.20015e+00   

library              JEFF-3.3  JENDL/FPD-2011  
decay nuclide                                  
beta  $^{137m}$Ba 1.78504e+00     0.00000e+00  
      $^{137}$Cs  9.35851e-01     9.00000e-01  
      $^{90}$Sr   5.00000e-01     8.00000e-01  
      $^{90}$Y    1.20015e+00     1.20000e+00  
gamma $^{137m}$Ba 1.32514e+00     1.00000e+00  
      $^{137}$Cs  2.26802e-04     0.00000e+00  
      $^{90}$Sr   0.00000e+00     0.00000e+00  
      $^{90}$Y    1.02541e-04     1.00000e-04  
total $^{137m}$Ba 2.22314e+00     1.00000e+00  
      $^{137}$Cs  9.35851e-01     9.00000e-01  
      $^{90}$Sr   5.00000e-01     8.00000e-01  
      $^{90}$Y    1.20015e+00     1.20000e+00

... and in relative units (%).

In [16]:
(uncert / energies * 100).fillna(0)

library            ENDF/B-VII.0  ENDF/B-VII.1  ENDF/B-VIII.0  JEFF-3.1.1  \
decay nuclide                                                              
beta  $^{137m}$Ba   1.14258e+00   1.09629e+00    1.09629e+00 2.93931e+00   
      $^{137}$Cs    3.61636e-01   3.60538e-01    3.60538e-01 4.81407e-01   
      $^{90}$Sr     0.00000e+00   4.08580e-01    4.08580e-01 2.87356e-01   
      $^{90}$Y      1.42752e-03   7.83398e-02    7.83398e-02 1.28521e-01   
gamma $^{137m}$Ba   6.66433e-02   1.55234e-01    1.55234e-01 2.21450e-01   
      $^{137}$Cs    1.37931e+01   1.37931e+01    1.37931e+01 1.37932e+01   
      $^{90}$Sr     0.00000e+00   0.00000e+00    0.00000e+00 0.00000e+00   
      $^{90}$Y      2.14286e+01   7.58698e+00    7.58698e+00 8.29244e+00   
total $^{137m}$Ba   1.26124e-01   1.75938e-01    1.75938e-01 3.37288e-01   
      $^{137}$Cs    3.61633e-01   3.60534e-01    3.60534e-01 4.81403e-01   
      $^{90}$Sr     0.00000e+00   4.08580e-01    4.08580e-01 2.87356e-01   
      $^{90}$Y      1.42752e-03   7.83397e-02    7.83397e-02 1.28521e-01   

library              JEFF-3.3  JENDL/FPD-2011  
decay nuclide                                  
beta  $^{137m}$Ba 2.93931e+00     0.00000e+00  
      $^{137}$Cs  5.20293e-01     4.81026e-01  
      $^{90}$Sr   2.58398e-01     4.08580e-01  
      $^{90}$Y    1.28521e-01     1.28507e-01  
gamma $^{137m}$Ba 2.21450e-01     1.68124e-01  
      $^{137}$Cs  1.37259e+01     0.00000e+00  
      $^{90}$Sr   0.00000e+00     0.00000e+00  
      $^{90}$Y    8.29244e+00     8.00000e+00  
total $^{137m}$Ba 3.37288e-01     1.51874e-01  
      $^{137}$Cs  5.20288e-01     4.81026e-01  
      $^{90}$Sr   2.58398e-01     4.08580e-01  
      $^{90}$Y    1.28521e-01     1.28507e-01

## Mean recoverable energy per decay chain

In [17]:
outputs = []
for lib in energies.columns:

    parent = 551370
    daughter = 561371
    chain = "$^{137}$Cs/$^{137m}$Ba"
    mode = '10' # decay mode: '10' = beta + gamma
    br = rdds[lib].data[parent]["decay_modes"][mode]['branching_ratio']
    for decay in ("gamma", "beta", "total"):
        out = {
            "decay": decay,
            "energy": rdds[lib].data[parent]['decay_energy'][decay] * 1e-3 +\
                      rdds[lib].data[daughter]['decay_energy'][decay] * 1e-3 * br,
            "library": lib,
            "chain": chain,
        }
        outputs.append(out)

    parent = 380900
    daughter = 390900
    chain = "$^{90}$Sr/$^{90}$Y"
    mode = '10' # decay mode: '10' = beta + gamma
    br = rdds[lib].data[parent]["decay_modes"][mode]['branching_ratio']
    for decay in ("gamma", "beta", "total"):
        out = {
            "decay": decay,
            "energy": rdds[lib].data[parent]['decay_energy'][decay] * 1e-3 +\
                      rdds[lib].data[daughter]['decay_energy'][decay] * 1e-3 * br,
            "library": lib,
            "chain": chain,
        }
        outputs.append(out)

chain_energies = pd.DataFrame(outputs).pivot_table(index=("decay", "chain"), columns="library", values="energy")
chain_energies

library                       ENDF/B-VII.0  ENDF/B-VII.1  ENDF/B-VIII.0  \
decay chain                                                               
beta  $^{137}$Cs/$^{137m}$Ba   2.40560e+02   2.40189e+02    2.40189e+02   
      $^{90}$Sr/$^{90}$Y       1.12941e+03   1.12881e+03    1.12881e+03   
gamma $^{137}$Cs/$^{137m}$Ba   5.65127e+02   5.65597e+02    5.65597e+02   
      $^{90}$Sr/$^{90}$Y       3.06074e-05   8.85767e-04    8.85767e-04   
total $^{137}$Cs/$^{137m}$Ba   8.05687e+02   8.05787e+02    8.05787e+02   
      $^{90}$Sr/$^{90}$Y       1.12941e+03   1.12882e+03    1.12882e+03   

library                       JEFF-3.1.1    JEFF-3.3  JENDL/FPD-2011  
decay chain                                                           
beta  $^{137}$Cs/$^{137m}$Ba 2.45200e+02 2.37174e+02     2.47367e+02  
      $^{90}$Sr/$^{90}$Y     1.10782e+03 1.12732e+03     1.12960e+03  
gamma $^{137}$Cs/$^{137m}$Ba 5.64882e+02 5.64642e+02     5.63276e+02  
      $^{90}$Sr/$^{90}$Y     1.23656e-03 1.23656e-03     1.25000e-03  
total $^{137}$Cs/$^{137m}$Ba 8.10081e+02 8.01816e+02     8.10643e+02  
      $^{90}$Sr/$^{90}$Y     1.10782e+03 1.12732e+03     1.12960e+03

### Uncertainty propagation

Create a dataframe with linear sensitivities and uncertainties on both branching ratios and recoverable energies.

In [18]:
uncertainties = {}
sensitivities = {}

for lib in energies.columns:

    parent = 551370
    daughter = 561371
    chain = "$^{137}$Cs/$^{137m}$Ba"
    mode = '10' # decay mode: '10' = beta + gamma
    for decay in ("gamma", "beta"):
        uncertainties[(lib, decay, chain, "br")] = rdds[lib].data[parent]["decay_modes"][mode]['branching_ratio_uncertainty']
        uncertainties[(lib, decay, chain, "energy_parent")] = rdds[lib].data[parent]['decay_energy_uncertainties'][decay]
        uncertainties[(lib, decay, chain, "energy_daughter")] = rdds[lib].data[daughter]['decay_energy_uncertainties'][decay]
        sensitivities[(lib, decay, chain, "br")] = rdds[lib].data[daughter]['decay_energy'][decay] * 1e-3
        sensitivities[(lib, decay, chain, "energy_parent")] = 1 * 1e-3
        sensitivities[(lib, decay, chain, "energy_daughter")] = rdds[lib].data[parent]["decay_modes"][mode]['branching_ratio'] * 1e-3

    parent = 380900
    daughter = 390900
    chain = "$^{90}$Sr/$^{90}$Y"
    mode = '10' # decay mode: '10' = beta + gamma
    for decay in ("gamma", "beta"):
        uncertainties[(lib, decay, chain, "br")] = rdds[lib].data[parent]["decay_modes"][mode]['branching_ratio_uncertainty']
        uncertainties[(lib, decay, chain, "energy_parent")] = rdds[lib].data[parent]['decay_energy_uncertainties'][decay]
        uncertainties[(lib, decay, chain, "energy_daughter")] = rdds[lib].data[daughter]['decay_energy_uncertainties'][decay]
        sensitivities[(lib, decay, chain, "br")] = rdds[lib].data[daughter]['decay_energy'][decay] * 1e-3
        sensitivities[(lib, decay, chain, "energy_parent")] = 1 * 1e-3
        sensitivities[(lib, decay, chain, "energy_daughter")] = rdds[lib].data[parent]["decay_modes"][mode]['branching_ratio'] * 1e-3

In [19]:
data_unc = pd.DataFrame([sensitivities, uncertainties], index=["sensitivity", "uncertainty"]).T
data_unc.index = pd.MultiIndex.from_tuples(data_unc.index, names=["library", "decay", "chain", "parameter"])
data_unc.reset_index(inplace=True)
data_unc

,library,decay,chain,parameter,sensitivity,uncertainty
0,ENDF/B-VII.0,gamma,$^{137}$Cs/$^{137m}$Ba,br,5.98649e+02,2.00000e-03
1,ENDF/B-VII.0,gamma,$^{137}$Cs/$^{137m}$Ba,energy_parent,1.00000e-03,2.26801e-01
2,ENDF/B-VII.0,gamma,$^{137}$Cs/$^{137m}$Ba,energy_daughter,9.44000e-04,3.98960e+02
3,ENDF/B-VII.0,beta,$^{137}$Cs/$^{137m}$Ba,br,6.43140e+01,2.00000e-03
4,ENDF/B-VII.0,beta,$^{137}$Cs/$^{137m}$Ba,energy_parent,1.00000e-03,6.50395e+02
...,...,...,...,...,...,...
67,JENDL/FPD-2011,gamma,$^{90}$Sr/$^{90}$Y,energy_parent,1.00000e-03,0.00000e+00
68,JENDL/FPD-2011,gamma,$^{90}$Sr/$^{90}$Y,energy_daughter,1.00000e-03,1.00000e-01
69,JENDL/FPD-2011,beta,$^{90}$Sr/$^{90}$Y,br,9.33800e+02,0.00000e+00
70,JENDL/FPD-2011,beta,$^{90}$Sr/$^{90}$Y,energy_parent,1.00000e-03,8.00000e+02


Propagate the uncertainties to the beta and gamma mean recoverable energies for the chain.

In [20]:
outputs = []
for (lib, chain, decay), df in data_unc.groupby(["library", "chain", "decay"]):
    variance = (df.sensitivity * df.uncertainty).apply(lambda x: x**2).sum()
    out = {
        "variance": variance,
        "uncertainty": np.sqrt(variance),
        "library": lib,
        "decay": decay,
        "chain": chain,
    }
    outputs.append(out)
uncertainties = pd.DataFrame(outputs)
uncertainties

,variance,uncertainty,library,decay,chain
0,9.20763e-01,9.59564e-01,ENDF/B-VII.0,beta,$^{137}$Cs/$^{137m}$Ba
1,1.57537e+00,1.25514e+00,ENDF/B-VII.0,gamma,$^{137}$Cs/$^{137m}$Ba
2,1.77624e-04,1.33276e-02,ENDF/B-VII.0,beta,$^{90}$Sr/$^{90}$Y
3,4.30169e-11,6.55873e-06,ENDF/B-VII.0,gamma,$^{90}$Sr/$^{90}$Y
4,8.78485e-01,9.37275e-01,ENDF/B-VII.1,beta,$^{137}$Cs/$^{137m}$Ba
5,2.19771e+00,1.48247e+00,ENDF/B-VII.1,gamma,$^{137}$Cs/$^{137m}$Ba
6,1.17425e+00,1.08363e+00,ENDF/B-VII.1,beta,$^{90}$Sr/$^{90}$Y
7,4.51624e-09,6.72030e-05,ENDF/B-VII.1,gamma,$^{90}$Sr/$^{90}$Y
8,8.78485e-01,9.37275e-01,ENDF/B-VIII.0,beta,$^{137}$Cs/$^{137m}$Ba
9,2.19771e+00,1.48247e+00,ENDF/B-VIII.0,gamma,$^{137}$Cs/$^{137m}$Ba


Propagate the uncertainties to the total recoverable energy for the chains.

In [21]:
series = uncertainties[["library", "chain", "variance"]].groupby(["library", "chain"]).sum()
total_uncertainties = series.assign(decay="total", uncertainty=np.sqrt(series.variance)).reset_index()
total_uncertainties

,library,chain,variance,decay,uncertainty
0,ENDF/B-VII.0,$^{137}$Cs/$^{137m}$Ba,2.49613e+00,total,1.57991e+00
1,ENDF/B-VII.0,$^{90}$Sr/$^{90}$Y,1.77624e-04,total,1.33276e-02
2,ENDF/B-VII.1,$^{137}$Cs/$^{137m}$Ba,3.07619e+00,total,1.75391e+00
3,ENDF/B-VII.1,$^{90}$Sr/$^{90}$Y,1.17425e+00,total,1.08363e+00
4,ENDF/B-VIII.0,$^{137}$Cs/$^{137m}$Ba,3.07619e+00,total,1.75391e+00
5,ENDF/B-VIII.0,$^{90}$Sr/$^{90}$Y,1.17425e+00,total,1.08363e+00
6,JEFF-3.1.1,$^{137}$Cs/$^{137m}$Ba,5.22226e+00,total,2.28523e+00
7,JEFF-3.1.1,$^{90}$Sr/$^{90}$Y,1.69036e+00,total,1.30014e+00
8,JEFF-3.3,$^{137}$Cs/$^{137m}$Ba,5.27635e+00,total,2.29703e+00
9,JEFF-3.3,$^{90}$Sr/$^{90}$Y,1.69036e+00,total,1.30014e+00


In absolute units (keV)...

In [22]:
absolute_unc = pd.concat([uncertainties, total_uncertainties])\
                 .pivot_table(index=["decay", "chain"], columns="library", values="uncertainty")
absolute_unc

library                       ENDF/B-VII.0  ENDF/B-VII.1  ENDF/B-VIII.0  \
decay chain                                                               
beta  $^{137}$Cs/$^{137m}$Ba   9.59564e-01   9.37275e-01    9.37275e-01   
      $^{90}$Sr/$^{90}$Y       1.33276e-02   1.08363e+00    1.08363e+00   
gamma $^{137}$Cs/$^{137m}$Ba   1.25514e+00   1.48247e+00    1.48247e+00   
      $^{90}$Sr/$^{90}$Y       6.55873e-06   6.72030e-05    6.72030e-05   
total $^{137}$Cs/$^{137m}$Ba   1.57991e+00   1.75391e+00    1.75391e+00   
      $^{90}$Sr/$^{90}$Y       1.33276e-02   1.08363e+00    1.08363e+00   

library                       JEFF-3.1.1    JEFF-3.3  JENDL/FPD-2011  
decay chain                                                           
beta  $^{137}$Cs/$^{137m}$Ba 1.91244e+00 1.92688e+00     9.08956e-01  
      $^{90}$Sr/$^{90}$Y     1.30014e+00 1.30014e+00     1.44222e+00  
gamma $^{137}$Cs/$^{137m}$Ba 1.25093e+00 1.25039e+00     1.52051e+00  
      $^{90}$Sr/$^{90}$Y     1.02541e-04 1.02541e-04     1.00000e-04  
total $^{137}$Cs/$^{137m}$Ba 2.28523e+00 2.29703e+00     1.77148e+00  
      $^{90}$Sr/$^{90}$Y     1.30014e+00 1.30014e+00     1.44222e+00

... and in relative units (%).

In [23]:
relative_unc = absolute_unc / chain_energies * 100
relative_unc

library                       ENDF/B-VII.0  ENDF/B-VII.1  ENDF/B-VIII.0  \
decay chain                                                               
beta  $^{137}$Cs/$^{137m}$Ba   3.98887e-01   3.90223e-01    3.90223e-01   
      $^{90}$Sr/$^{90}$Y       1.18004e-03   9.59969e-02    9.59969e-02   
gamma $^{137}$Cs/$^{137m}$Ba   2.22098e-01   2.62106e-01    2.62106e-01   
      $^{90}$Sr/$^{90}$Y       2.14286e+01   7.58698e+00    7.58698e+00   
total $^{137}$Cs/$^{137m}$Ba   1.96095e-01   2.17664e-01    2.17664e-01   
      $^{90}$Sr/$^{90}$Y       1.18004e-03   9.59968e-02    9.59968e-02   

library                       JEFF-3.1.1    JEFF-3.3  JENDL/FPD-2011  
decay chain                                                           
beta  $^{137}$Cs/$^{137m}$Ba 7.79954e-01 8.12432e-01     3.67452e-01  
      $^{90}$Sr/$^{90}$Y     1.17361e-01 1.15331e-01     1.27675e-01  
gamma $^{137}$Cs/$^{137m}$Ba 2.21449e-01 2.21449e-01     2.69941e-01  
      $^{90}$Sr/$^{90}$Y     8.29244e+00 8.29244e+00     8.00000e+00  
total $^{137}$Cs/$^{137m}$Ba 2.82098e-01 2.86478e-01     2.18528e-01  
      $^{90}$Sr/$^{90}$Y     1.17360e-01 1.15330e-01     1.27675e-01